In [1]:
import sys
import os
sys.path.append('../')

In [2]:
import segmentation_models_pytorch as smp
from scripts.preprocessing import *
from torch.utils.data import DataLoader, TensorDataset
import torch

In [3]:
NUM_CPU = 2 # os.cpu_count()
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

TRAINING_SIZE = 100
BATCH_SIZE = 16
NUM_EPOCHS = 10
BASE_LR = 0.01

ENCODER = 'resnet34'
ENCODER_WEIGHTS = 'imagenet'
ACTIVATION = 'sigmoid'

In [4]:
train_dir = "../dataset/training/"
train_images_filename = train_dir + "images/"
train_masks_filename = train_dir + "groundtruth/"

train_images = extract_data(train_images_filename, TRAINING_SIZE)
train_masks = extract_data(train_masks_filename, TRAINING_SIZE)
print(f"Input shapes: {train_images.shape, train_masks.shape}")

Loading ../dataset/training/images/satImage_001.png
Loading ../dataset/training/images/satImage_100.png
Loading ../dataset/training/groundtruth/satImage_001.png
Loading ../dataset/training/groundtruth/satImage_100.png
Input shapes: ((100, 400, 400, 3), (100, 400, 400))


In [5]:
# Resize images to a size divisible by 32 to make it UNet compatible
train_data = img_resize(np.transpose(train_images, (0, 3, 1, 2)), (416, 416))
train_labels = img_resize(np.transpose(np.expand_dims(train_masks, -1), (0, 3, 1, 2)),  (416, 416))
print(f"New shapes: {train_data.shape, train_labels.shape}")

New shapes: (torch.Size([100, 3, 416, 416]), torch.Size([100, 1, 416, 416]))


In [6]:
# Create segmentation model with pretrained encoder
model = smp.Unet(
    encoder_name=ENCODER, 
    encoder_weights=ENCODER_WEIGHTS, 
    classes=1, 
    activation=ACTIVATION,
)

model = model.to(DEVICE)
# preprocessing_fn = smp.encoders.get_preprocessing_fn(ENCODER, ENCODER_WEIGHTS)

In [7]:
# Create dataset
road_dataset = TensorDataset(train_data, train_labels)

# Get train and val data loaders
train_loader = DataLoader(road_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=NUM_CPU)

In [8]:
# define loss
criterion = smp.losses.DiceLoss('binary')

# define optimizer
optimizer = torch.optim.Adam(model.parameters(), lr=BASE_LR)

# define learning rate scheduler (not used in this NB)
lr_scheduler = torch.optim.lr_scheduler.CosineAnnealingWarmRestarts(
    optimizer, T_0=1, T_mult=2, eta_min=5e-5,
)

In [9]:
def train_epoch(model, optimizer, scheduler, criterion, train_loader, epoch, device):
    # ***************************************************
    # Set model to training mode (affects dropout, batch norm e.g.)
    model.train()

    loss_history = []
    accuracy_history = []
    f1_history = []
    lr_history = []

    for batch_idx, (data, target) in enumerate(train_loader):
        # Move the data to the device
        print(data.shape, target.shape)
        data, target = data.to(device), target.to(device)
        # Zero the gradients
        optimizer.zero_grad()
        # Compute model output
        output = model(data)
        # Compute loss
        print(output.shape, target.shape)
        print(output[0])
        loss = criterion(output, target)
        # Backpropagate loss
        loss.backward()
        # Perform an optimizer step
        optimizer.step()

        # Perform a learning rate scheduler step
        scheduler.step()
        # Compute accuracy_float (float value, not a tensor)
        pred = output.argmax(dim=1, keepdim=True)
        correct = pred.eq(target.view_as(pred)).sum().item()
        accuracy_float = correct / len(data)
        # Compute loss_float (float value, not a tensor)
        loss_float = loss.item()
        # Add loss_float to loss_history
        loss_history.append(loss_float)
        # Add accuracy_float to accuracy_history
        accuracy_history.append(accuracy_float)
        # Add learning rate to lr_history
        lr_history.append(scheduler.get_last_lr()[0])

        if batch_idx % (len(train_loader.dataset) // len(data) // 10) == 0:
            print(
                f"Train Epoch: {epoch}-{batch_idx:03d} "
                f"batch_loss={loss_float:0.2e} "
                f"batch_acc={accuracy_float:0.3f} "
                f"lr={scheduler.get_last_lr()[0]:0.3e} "
            )

    return loss_history, accuracy_history, lr_history    

In [10]:
train_epoch(model, optimizer, lr_scheduler, criterion, train_loader, 0, DEVICE)

torch.Size([16, 3, 416, 416]) torch.Size([16, 1, 416, 416])
